In [ ]:
import tensorflow.keras.backend as K
import tensorflow.keras as keras
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
import numpy as np
import scipy as sp
import librosa
import librosa.display
import soundfile as sf
import pandas as pd
import os
from pathlib import *
import glob
import shutil
import IPython.display as ipd
from ipywidgets import interact, fixed, FloatSlider
%pylab
%matplotlib inline

In [ ]:
INPUT_PATH = Path('/net/store/cbc/projects/Pan troglodytes/audio_PH_dataset/PH/good_data/')
file_path =  '/net/store/cv/users/ybrima/scratch/data/Luango_Speaker/clip_loango.npz'
BASE_PATH =  '/net/store/cv/users/ybrima/scratch/data/Luango_Speaker/'
CLASSES = os.listdir(INPUT_PATH)
FRAME_LENGHT =  1024
SAMPLE_RATE =  44100
HOP_LENGTH =  512

In [ ]:
def get_files(INPUT_PATH):
    ds =  {'file': [], 'class': [], 'label':  [], 'duration': []}
    for d in os.listdir(INPUT_PATH):
        temp =  Path(INPUT_PATH, d)
        if(os.path.isdir(temp)):
            for file in temp.glob("**/*.wav"):
                filename =  Path(temp,file)
                y,_ = librosa.load(filename, sr=SAMPLE_RATE)
                TRACK_DURATION = (1/SAMPLE_RATE)* len(y) # measured in seconds
                ds['file'].append(filename)
                ds['class'].append(CLASSES.index(d))
                ds['label'].append(d)
                ds['duration'].append(TRACK_DURATION)
    data = pd.DataFrame(ds)
    data.to_csv(Path(BASE_PATH,'metadata.csv'), index=False)
    print(f"{data.shape[0]} files read successfully")
    return data

In [ ]:
df = get_files(INPUT_PATH)

In [ ]:
df['class'].value_counts()

In [ ]:
file = df.sample(n=1).iloc[0,0]
ipd.Audio(file)

In [ ]:
file1 = df.iloc[0,0]
file2 = df.iloc[1,0]
file3 = df.iloc[2,0]

In [ ]:
y,sr =  librosa.load(file1,sr=None)
y1,sr =  librosa.load(file2,sr=None)
y2,sr =  librosa.load(file3,sr=None)

In [ ]:
print(len(y) * (1/sr))
print(len(y1) * (1/sr))
print(len(y2) * (1/sr))

In [ ]:
df.head()

In [ ]:
max_idx = df[df['duration'] == df['duration'].max()].index
min_idx = df[df['duration'] == df['duration'].min()].index
df.iloc[max_idx]

In [ ]:
df.iloc[min_idx]

In [ ]:
yy,sr =  librosa.load(df.iloc[max_idx[0],0],sr=None)

In [ ]:
y,sr =  librosa.load(df.iloc[min_idx[0],0],sr=None)

In [ ]:
def init_outfiles():
    CLEAN_PATH =  Path(BASE_PATH,'good_data')
    if(not os.path.exists(CLEAN_PATH)):
        os.mkdir(CLEAN_PATH)
        print(f'Base directory successfully created at {CLEAN_PATH}')
    for d in os.listdir(INPUT_PATH):
        if(not os.path.exists(Path(CLEAN_PATH,d))):
            os.mkdir(Path(CLEAN_PATH,d))
            print(f'Base directory successfully created at {Path(CLEAN_PATH,d)}')
    print(f'Process completed successfully {len(os.listdir(INPUT_PATH))} directories created')


In [ ]:
CLEAN_PATH =  Path(BASE_PATH,'good_data')

In [ ]:
def get_files_updated(INPUT_PATH):
    df = get_files(INPUT_PATH)
    max_idx = df[df['duration'] == df['duration'].max()].index
    min_idx = df[df['duration'] == df['duration'].min()].index
    yy,sr =  librosa.load(df.iloc[max_idx[0],0],sr=None)


    ds =  {'file': [], 'class': [], 'label':  [], 'duration': []}
    for d in os.listdir(INPUT_PATH):
        temp =  Path(INPUT_PATH, d)
        out_temp = Path(CLEAN_PATH, d)
        if(os.path.isdir(temp)):
            for file in temp.glob("**/*.wav"):
                filename =  Path(temp,file)
                outfile = Path(out_temp,file.name)
                y,sr = librosa.load(filename, sr=SAMPLE_RATE)
                dff =  int(np.ceil( len(yy) - len(y) ) / 2)
                a  = np.lib.pad(y, (int(np.floor(dff)) ,), 'symmetric')
                TRACK_DURATION = (1/SAMPLE_RATE)* len(a) # measured in seconds
                sf.write(outfile,a,sr)
                ds['file'].append(outfile)
                ds['class'].append(CLASSES.index(d))
                ds['label'].append(d)
                ds['duration'].append(TRACK_DURATION)
    data = pd.DataFrame(ds)
    data.to_csv(Path(BASE_PATH,'metadata_clean.csv'), index=False)
    print(f"{data.shape[0]} files read successfully")
    return data

In [ ]:
df = get_files_updated(INPUT_PATH)

In [ ]:
df.head()

In [ ]:
plt.figure(figsize=(12,6))
plt.subplot(121)
librosa.display.waveplot(y,sr=sr)
plt.title("Shortest Recording")
plt.subplot(122)
librosa.display.waveplot(yy,sr=sr)
plt.title("Longest Recording")
plt.show()

In [ ]:
q = df.sample(n=1)
q

In [ ]:
q,sr =  librosa.load(q.iloc[0,0],sr=None)

In [ ]:
dff =  int(np.ceil( len(yy) - len(q) ) / 2)
a =  q

In [ ]:
a  = np.lib.pad(a, (int(np.floor(dff)) ,), 'symmetric')

In [ ]:
def process_files(df):
    for k in range(df.shape[0]):

In [ ]:
librosa.display.waveplot(a,sr=sr)

In [ ]:
plt.figure(figsize=(12,6))
plt.subplot(121)
librosa.display.waveplot(a,sr=sr)
plt.title("Shortest Recording")
plt.subplot(122)
librosa.display.waveplot(yy,sr=sr)
plt.title("Longest Recording")
plt.show()

In [ ]:
sf.write(Path(BASE_PATH,'symmetric_padding.wav'),a,sr)

In [ ]:
df['label'].value_counts()

In [ ]:
y,_ =  librosa.load(file,sr=SAMPLE_RATE)

In [ ]:
librosa.display.waveplot(y)

In [ ]:
samples = df.sample(n=16,replace=False)
fig =  plt.figure(1,figsize=(24,14))
for k in range(samples.shape[0]):
    r =  df.iloc[k]
    file = r[0]
    y,_ = librosa.load(file,sr=SAMPLE_RATE)
    ax =  fig.add_subplot(4,4,k+1)
    librosa.display.waveplot(y, sr=sr)
    ax.set_title(f"{r[2]}  {str(file.name)}")
plt.tight_layout()
plt.show()

In [ ]:
y, sr = librosa.load(librosa.util.example_audio_file())


In [ ]:
file

In [ ]:
y,_ = librosa.load(file,sr=SAMPLE_RATE,duration=17)

In [ ]:
out_file ='/net/store/cv/users/ybrima/scratch/data/EDA/clip21_loango_updated.wav'

In [ ]:
import noisereduce as nr
reduced_noise = nr.reduce_noise(y=y, sr=SAMPLE_RATE)

In [ ]:
librosa.display.waveplot(reduced_noise)

In [ ]:
ft =  sp.fft.fft(y)
magnitude =  abs(ft)
freq =  np.linspace(0,SAMPLE_RATE,len(magnitude))


In [ ]:
# plot spectrum

# Next we want to visualize the rmse for the  audio wave 

plt.figure(1,figsize=(18,8))
plt.plot(freq,magnitude)
plt.xlabel("Frequency (Hz)")
plt.ylabel("Magnitude")
plt.xticks(np.arange(min(freq), max(freq)+1, 1500.0))
plt.xlim([0,max(freq)])
plt.title("Fast Fourier Transform")
# plt.savefig(f'{figpath}/FFT.pdf', dpi=300)
plt.show()


In [ ]:
# yy.shape

In [ ]:
from scipy import signal

In [ ]:
b = signal.firwin(101, cutoff=1000, fs=SAMPLE_RATE, pass_zero=False)
x = signal.lfilter(b, [1.0], y)


In [ ]:
ft =  sp.fft.fft(x)
magnitude =  abs(ft)
freq =  np.linspace(0,SAMPLE_RATE,len(magnitude))

# plot spectrum

# Next we want to visualize the rmse for the  audio wave 

plt.figure(1,figsize=(18,8))
plt.plot(freq,magnitude)
plt.xlabel("Frequency (Hz)")
plt.ylabel("Magnitude")
plt.xticks(np.arange(min(freq), max(freq)+1, 1500.0))
plt.xlim([0,max(freq)])
plt.title("Fast Fourier Transform")
# plt.savefig(f'{figpath}/FFT.pdf', dpi=300)
plt.show()

In [ ]:
file_path =  '/net/store/cv/users/ybrima/scratch/data/Luango_Speaker/clip_loango.npz'

In [ ]:
X  = []
y =  []
num_segments = 5
num_mfcc= 32
n_fft=2048
FRAME_LENGHT =  2048

for k in range(df.shape[0]):
    r =  df.iloc[k]
    
    #   Loading the audio file 
    yy,sample_rate = librosa.load(r['file'], sr=SAMPLE_RATE)
    TRACK_DURATION = (1/SAMPLE_RATE)* len(signal) # measured in seconds
    SAMPLES_PER_TRACK = SAMPLE_RATE * TRACK_DURATION
    
    
    for i in range(0,len(yy),HOP_LENGTH):
        signal = yy[i:i+samples_per_segment]
        if(len(signal) == samples_per_segment):
            # extract mfcc
            D = librosa.amplitude_to_db(np.abs(librosa.stft(signal)), ref=np.max)
            X.append(D)
            y.append(CLASSES.index(r['label']))

X =  np.array(X)
y =  np.array(y)
np.savez(file_path,x=X,y=y)
print(f"Data written to storage successfully, path = {file_path}")

In [ ]:
sf.write(out_file,reduced_noise,SAMPLE_RATE)

In [ ]:
# frames =  range(0,y.size)
# t  = librosa.frames_to_time(frames,hop_length=HOP_LENGTH)


import tensorflow as tf
from tensorflow.keras.layers import Conv1D,Conv1DTranspose,Concatenate,Input